## 언어적 평가 AI 모델(KLUE-BERT) - 유효성 검증 

### 1단계: 필요 라이브러리 설치 및 준비

In [2]:
!python3 --version

Python 3.8.10


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
import torch
torch.__version__

# 2.0.1+cu117

'2.0.1+cu117'

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [6]:
import tensorflow as tf

print(tf.__version__)
# 2.13.0

2023-11-27 15:35:38.176120: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.13.0


In [7]:
import keras

print(keras.__version__)


2.13.1


In [8]:
%pip install nvidia-cudnn-cu11==8.6.0.163


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
print(torch.backends.cudnn.version())

8600


In [10]:
%pip install transformers


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install seqeval


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf


### 2단계: 데이터 준비 - 라벨링 데이터 로드(load)

In [15]:
# 라벨링 데이터 가져오는 함수
def get_data(data, path):
    file_name, presen_script, script_stt_txt, script_tag_txt = [], [], [], []  # 빈 리스트 생성 (6개)
    for row in tqdm(data.itertuples(), total=data.shape[0]):  # 진행상황 확인을 위한 tqdm, data frame 값을 빠르게 가져오기 위한 itertuples
        file_str = path + row.file_name                       # 파일명

        with open(file_str) as f:      # json파일 열기
            text = json.load(f)

        file_name.append(text['info']['filename'])  # filename
        presen_script.append(text['presentation']['presen_script'])    # 발화원문
        script_stt_txt.append(text['script']['script_stt_txt'])   # 발화 내용(STT)
        script_tag_txt.append(text['script']['script_tag_txt'])    # 발화 내용(태그매핑)

    df= pd.DataFrame({'file_name': file_name, 'presen_script': presen_script, 'script_stt_txt': script_stt_txt, 'script_tag_txt': script_tag_txt})  # 리스트로 dataframe 만들기
    return df

In [16]:
import pandas as pd
from os import listdir
from tqdm import tqdm
import json

 # [주의] dataset폴더 안에 라벨링데이터(json) 파일 저장
paths = 'dataset/'
fileNameList = listdir(paths)
len(fileNameList)

main_df = pd.DataFrame()

df= pd.DataFrame(fileNameList, columns=['file_name'])     # 파일 리스트 -> 데이터 프레임화
df=df.astype('string')                                    # df type string 으로 변경

cr = df['file_name'].str.contains('A') # presentation
data = df[cr]

sub_df = get_data(data, paths)                          # 데이터 가져오는 함수
main_df = pd.concat([main_df, sub_df])            # main_labels에 누적하여 더하기
main_df.head(5)

10-태그인식_데이터전처리코드_v0.1.ipynb
10-태그인식_데이터전처리코드_v0.2.ipynb
10-태그인식_데이터전처리코드_v0.3.ipynb
20-KLUE_BERT를_이용한_태그_인식_customizing_v0.1.ipynb
20-KLUE_BERT를_이용한_태그_인식_customizing_v0.12.ipynb
20-KLUE_BERT를_이용한_태그_인식_customizing_v0.13.ipynb
20-KLUE_BERT를_이용한_태그_인식_customizing_v0.3.ipynb
20-KLUE_BERT를_이용한_태그_인식_valid_customizing_v0.11.ipynb
Dockerfile
KLUE_BERT를_이용한_태그_인식_customizing_v0.14.ipynb
NER
dataset
dataset_2309
dataset_2310_556e
dataset_2310_636e
docker
docker-compose.yml
docker_lang.tar
klue-bert-pytorch.ipynb
klue-bert.py
logs
main_df_csv_800e.csv
ner_label_v2.txt
nia89_lang_workspace.tar
nia89_workspace
nia89_workspace_test
old
old_data
requirements.txt
results
sample_docker_test
saved_model
summarybot
tag_real_test_data_100.csv
tag_real_test_data_11.csv
tag_real_test_data_13_norandom.csv
tag_real_test_data_99.csv
tag_real_train_data_100.csv
tag_real_train_data_11.csv
tag_real_train_data_13_norandom.csv
tag_real_train_data_99.csv
tag_test_data.csv
tag_test_data_2155e.csv
tag_test_data_216

100%|██████████| 800/800 [00:00<00:00, 2387.51it/s]


,file_name,presen_script,script_stt_txt,script_tag_txt
0,A01_S02_M_A_08_019_02_WA_MO_presentation.json,과학의 발전으로 현대인의 평균 수명이 지속적으로 증가하고 있는 현상은 인류에게 큰 ...,과학의 발전으로 현대인의 평균 수명이 지속적으로 증가하고 있는현상을인류에게 큰 변화...,과학의 발전으로 현대인의 평균 수명이 지속적으로 증가하고 있는 <WR>현상을</WR...
1,A05_S02_F_F_01_092_02_WA_MO_presentation.json,"지적 재산권은 현대 사회에서 매우 중요한 개념으로 여겨지며, 기술과 창의성의 보호 ...",지적 재산권은 현대 사회에서 매우 중요한 개념으로 여겨지며 기술과 창의성의 보호 및...,지적 재산권은 현대 사회에서 매우 중요한 개념으로 여겨지며 기술과 창의성의 보호 및...
2,A03_S03_M_F_05_100_02_WA_MO_presentation.json,비혼주의는 혼인을 피하고 혼자 또는 동반자와 함께 삶을 즐기는 사상이나 생활 방식을...,비혼주의는 혼인을 피하고 혼자 또는 동반자와 함께 삶은 즐기는 사상이나 생활 방식을...,비혼주의는 혼인을 피하고 혼자 또는 동반자와 함께 <WR>삶은</WR> 즐기는 ...
3,A01_S01_F_C_01_045_02_WA_MO_presentation.json,"안녕하세요, 여러분. 오늘은 저의 미래에 대해 이야기하고자 합니다. 제가 가지고 있...","안녕하세요, 여러분. 오늘은 저의 미래에 대해 이야기하고자 합니다.제가 가지고 있는...","안녕하세요, 여러분. 오늘은 저의 미래에 대해 이야기하고자 합니다.제가 가지고 있는..."
4,A01_S02_M_A_04_011_02_WA_MO_presentation.json,"안녕하세요, 자동차 및 운송수단의 발전이 우리의 삶에 미치는 부정적인 영향에 대해 ...",안녕하세요. 자동차 및 운송수단의 발전이 우리의 삶에 영향이 미치는 부정적인 영향에...,안녕하세요. 자동차 및 운송수단의 발전이 우리의 삶에 영향이 미치는 부정적인 영향에...


In [16]:
# 모든 라벨링데이터셋을 CSV 파일로 저장
main_df_csv = pd.DataFrame(main_df, columns=["file_name", "presen_script", "script_stt_txt", "script_tag_txt"])
main_df_csv.to_csv("main_df_csv_800e.csv", encoding="utf-8-sig", index=False)


### 3단계: 데이터 가공 - 태그 시퀀스 레이블링 

In [334]:
import re

def label_text(text):
    labels = []
    words = text.split()
    
    for word in words:
        # 개체명 인식
        if  word.startswith("<") or word.endswith("/>"):
            labels.append(word)
        elif word.startswith("자") and word.endswith(">"):  # (예외:태그구문규칙 불일치) 자신의</REP><WR>이미지</WR><WR>강조하고</WR><WR>있는</WR> -> 자신의</REP> <WR>이미지</WR> <WR>강조하고</WR> <WR>있는</WR>
            labels.append(word)
        elif word == '다양성과<FIL>': # (예외:태그구문규칙 불일치) 다양성과<FIL> 포용을 </FIL>훼손하며 -> 다양성과 <FIL>포용을</FIL> 훼손하며
            labels.append(word)
        # 개체명이 아닌 경우 O 태그 부여
        else:
            labels.append('O')

    return labels
 
def merge_tags(text):
    text = re.sub(r'\*', '', text.strip()) # *O
    text = re.sub(r'\*\w', 'O', text.strip()) # *O
    text = re.sub(r'<REP>\w\*\w', 'REP-B', text.strip())
    text = re.sub(r'<WR>\w+</WR><PS/><WR>\w+</WR>', 'WR-B PS-B WR-B', text.strip()) 
    text = re.sub(r'<PS/><WR>\w+</WR>', 'PS-B WR-B', text.strip()) 
    text = re.sub(r'<FIL>\w</FIL><PS/><FIL>\w</FIL>', 'FIL-B PS-B FIL-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR></REP><FIL>\w</FIL>', 'WR-B FIL-B', text.strip()) 
    text = re.sub(r'<PS/><FIL>\w</FIL>', 'PS-B FIL-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR><PS/><WR>\w+</WR>', 'WR-B PS-B WR-B', text.strip()) 
    text = re.sub(r'<REP>\w<FIL>\w</FIL>\w+</REP>', 'REP-B FIL-B O', text.strip()) 
    text = re.sub(r'<FIL>\w</FIL>\w+<FIL>\w</FIL><REP>\w', 'FIL-B O FIL-B REP-B', text.strip()) 
    text = re.sub(r'<REP>\w+<FIL>\w+</FIL>', 'REP-B FIL-B', text.strip()) 
    text = re.sub(r'<FIL>\w+</FIL><PS/><FIL>\w+</FIL>', 'FIL-B PS-B FIL-B ', text.strip()) 
    text = re.sub(r'<REP><WR>\w+</WR><FIL>\w+</FIL>', 'REP-B WR-B FIL-B ', text.strip()) 
    text = re.sub(r'<WR>\w+</WR></REP>\w+<WR></WR><FIL>\w+</FIL><REP>\w+', 'WR-B O WR-B FIL-B REP-B', text.strip()) 
    text = re.sub(r'<WR>\w+(-)\w+</WR>', 'WR-B', text.strip())
    text = re.sub(r'<REP><WR>\w+</WR><FIL>\w+</FIL>\w+</REP>', 'REP-B WR-B FIL-B O', text.strip()) 
    text = re.sub(r'<WR>\w+</WR></REP><WR>\w+</WR>', 'WR-B WR-B', text.strip()) 
    text = re.sub(r'<REP>\w+<WR>\w+</WR></REP><WR>\w+</WR><REP>\w+</REP>', 'REP-B WR-B WR-B REP-B', text.strip())
    text = re.sub(r'<REP>\w+<WR>\w+</WR></REP><WR>\w+</WR>', 'REP-B WR-B WR-B', text.strip()) 
    text = re.sub(r'<REP>\w+<WR>\w+</WR>\w+', 'REP-B WR-B O', text.strip()) 
    text = re.sub(r'<FIL>\w+</FIL><WR>\w+</WR></REP>', 'FIL-B WR-B', text.strip()) 
    text = re.sub(r'<FIL>\w+</FIL><WR>\w+</WR><REP>\w', 'FIL-B WR-B REP-B', text.strip()) 
    text = re.sub(r'<REP>\w+</REP><REP>\w+<WR>\w+</WR></REP>', 'REP-B REP-B WR-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR>\w+<WR>\w+</WR><REP>\w+', 'WR-B O WR-B REP-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR>\w+<WR>\w+</WR><WR>\w+</WR>', 'WR-B O WR-B WR-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR><FIL>\w+</FIL>\w+', 'WR-B FIL-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR><WR>\w+</WR>\w+<WR>\w+</WR>', 'WR-B WR-B O WR-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR></REP>\w+<WR></WR>', 'WR-B O WR-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR></REP>', 'WR-B O', text.strip()) 
    text = re.sub(r'<WR>\w+</WR><WR>\w+</WR><WR>\w+</WR><REP>\w+<WR>\w+</WR>', 'WR-B WR-B WR-B REP-B WR-B', text.strip())
    text = re.sub(r'<WR>\w+</WR><WR>\w+</WR><REP>\w+', 'WR-B WR-B REP-B', text.strip()) 
    text = re.sub(r'<REP><WR>\w+</WR>', 'REP-B WR-B', text.strip()) 
    text = re.sub(r'<REP>\w+<WR>\w+</WR>', 'REP-B WR-B', text.strip()) 
    text = re.sub(r'\w+</WR></REP> ', 'O', text.strip()) 
    text = re.sub(r'<REP>\w+<WR>\w+', 'REP-B WR-B', text.strip())
    text = re.sub(r'<PS/><WR></WR><FIL>\w+</FIL>', 'PS-B WR-B FIL-B', text.strip())
    text = re.sub(r'<WR><REP>\w+', 'WR-B REP-B', text.strip())
    text = re.sub(r'<WR>\w+</WR><PS/>', 'WR-B PS-B', text.strip())
    text = re.sub(r'<WR>\w+</WR><REP>\w+</REP><WR>\w+</WR>', 'WR-B REP-B WR-B', text.strip()) 
    text = re.sub(r'<FIL>\w+</FIL>\w+</REP>\w+<FIL>\w+</FIL><WR>\w+</WR>', 'FIL-B O FIL-B WR-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR><REP>\w', 'WR-B REP-B', text.strip()) 
    text = re.sub(r'<REP>\w+<WR>\w+</WR></REP><WR>\w+</WR><REP>\w+</REP>', 'REP-B WR-B WR-B REP-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR><WR>\w+</WR><REP>\w+', 'WR-B WR-B REP-B', text.strip()) 
    text = re.sub(r'<WR>\w+</WR><WR>\w+</WR><WR>\w+</WR><REP>\w+<WR>\w+</WR>', 'WR-B WR-B WR-B REP-B WR-B', text.strip())
    text = re.sub(r'<REP>\w+<WR>\w+</WR>\w+', 'REP-B WR-B O', text.strip()) 
    text = re.sub(r'<WR>(.*?)</WR><REP>(.*?)</REP><WR>(.*?)</WR>', 'WR-B REP-B WR-B', text.strip()) 
    text = re.sub(r'<REP>\w+</REP><WR>\w+<WR>\w+</WR></WR><REP>\w+', 'REP-B WR-B WR-B REP-B', text.strip()) 
    text = re.sub(r'<FIL>\w+</FIL><WR>\w+</WR><WR>\w+</WR><WR>\w+</WR>', 'FIL-B WR-B WR-B WR-B', text.strip())
    text = re.sub(r'<FIL>\w+<FIL/><PS/><FIL>\w+</FIL>', 'FIL-B PS-B FIL-B', text.strip())
    text = re.sub(r'<FIL>\w+</FIL><PS/>', 'FIL-B PS-B', text.strip())
    text = re.sub(r'<<FIL>\w+</FIL>', 'FIL-B', text.strip())
    text = re.sub(r'\w+<PS/>', 'PS-B', text.strip())
    text = re.sub(r'<PS/>', 'PS-B', text.strip())
    text = re.sub(r'<PS/>\w+', 'PS-B O', text.strip())
    text = re.sub(r'PS-B\w+', 'PS-B O', text.strip())
    text = re.sub(r'\w+<FIL>', 'FIL-B', text.strip()) 
    text = re.sub(r'<FIL>\w+</FIL>\w+', 'FIL-B O', text.strip())
    text = re.sub(r'<FIL>\s*(.*?)\s*</FIL>', r' <FIL>\1</FIL> ', text.strip())
    text = re.sub(r'<FIL>(.*?)</FIL>', 'FIL-B', text.strip())
    text = re.sub(r'<FIL>(.*?) ', 'FIL-B ', text.strip())
    text = re.sub(r'<FIL>', 'FIL-B', text.strip())
    text = re.sub(r'</FIL>\w+', 'O', text.strip()) 
    text = re.sub(r"<FIL>\w+|FIL\w+|FIL \w+|FIL,", "FIL-B", text.strip()).replace(",", "")
    text = re.sub(r'<REP>\w+(")\w+</REP>', 'REP-B', text.strip()) 
    text = re.sub(r'<REP>\w+</REP>', 'REP-B', text.strip())
    text = re.sub(r'<REP><WR>\w+', 'REP-B WR-B', text.strip()) 
    text = re.sub(r'<REP>\w+-\w+', 'REP-B WR-B', text.strip()) 
    text = re.sub(r'</REP><REP>\w+', 'REP-B', text.strip()) 
    text = re.sub(r'</REP>자', 'REP-B', text.strip()) 
    text = re.sub(r'</REP>창', 'REP-B', text.strip()) 
    text = re.sub(r'<REP>\w+', 'REP-B', text.strip()) 
    text = re.sub(r'<REP>\w+<WR>\w+</WR></REP><WR>\w+</WR>', 'REP-B WR-B WR-B', text.strip()) 
    text = re.sub(r'<REP>(.*?)</REP>', 'REP-B', text.strip())
    text = re.sub(r'<REP>\w+ <PS/>', 'REP-B PS-B', text.strip()) 
    text = re.sub(r'\w+</REP><WR>\w+</WR><WR>\w+</WR><WR>\w+</WR>', 'O WR-B WR-B WR-B', text.strip()) 
    text = re.sub(r'<REP>\w+</REP><REP>\w+<WR>\w+</WR></REP>', 'REP-B REP-B WR-B', text.strip()) 
    text = re.sub(r'<REP>[가-힣?]+', 'REP-B', text.strip())
    text = re.sub(r'<REP> <REP>', 'REP-B', text.strip())
    text = re.sub(r'<REP>[A-Za-z0-9_-가-힣]+', 'REP-B', text.strip()) 
    text = re.sub(r'REP-B\w+', 'REP-B ', text.strip())
    text = re.sub(r'<REP>\s*(.*?)\s*</REP>', r' <REP>\1</REP> ', text.strip())
    text = re.sub(r'<REP>(.*?)</REP>', 'REP-B', text.strip())
    text = re.sub(r"<REP>\w+|REP\w+|REP \w+|REP,", "REP-B", text).replace(",", "")
    text = re.sub(r'\w+<REP>', 'REP-B', text.strip())
    text = re.sub(r'</REP>\w+','O', text.strip()) 
    text = re.sub(r'\w+</REP>','O', text.strip()) 
    text = re.sub(r'</REP>','', text.strip())
    text = re.sub(r'<REP>', 'REP-B', text.strip()) 
    text = re.sub(r'<R','', text.strip())
    text = re.sub(r'<WR>\w+</WR><WR>\w+</WR>', 'WR-B WR-B', text.strip()) 
    text = re.sub(r'</WR></WR>\w+', 'O', text.strip()) 
    text = re.sub(r'</WR>\w+', 'O', text.strip())
    text = re.sub(r'\w+</WR>', 'O', text.strip())
    text = re.sub(r'</WR>', 'O', text.strip())
    text = re.sub(r'<WR>\w+</WR><WR>\w+</WR><WR>\w+</WR>', 'WR-B WR-B WR-B', text.strip())
    text = re.sub(r'<WR>\w+</WR>\w+', 'WR-B O', text.strip())
    text = re.sub(r'<WR>\w+</WR>(.*?)', 'WR-B', text.strip())
    text = re.sub(r'<WR>\w+', 'WR-B', text.strip())
    text = re.sub(r'<WR>\s*(.*?)\s*</WR>', r' <WR>\1</WR> ', text.strip())
    text = re.sub(r'<WR>(.*?)</WR>', 'WR-B', text.strip())
    text = re.sub(r'<WR>(.*?)</WR></REP>', 'WR-B', text.strip()) 
    text = re.sub(r'<WR>(.*?)</WR>(.*?)', 'WR-B', text.strip()) 
    text = re.sub(r'<WR>(.*?) ', 'WR-B ', text.strip())
    text = re.sub(r'</WR>(.*?)</WR>', 'WR-B', text.strip()) 
    text = re.sub(r'</WR>','', text.strip())
    text = re.sub(r'</WR></WR>\w+', 'WR-B', text.strip()) 
    text = re.sub(r'</WR></WR>', '', text.strip()) 
    text = re.sub(r'</WR>\w+</WR>', 'WR-B', text.strip()) 
    text = re.sub(r"<WR>\w+|WR\w+|WR \w+|WR,", "WR-B", text.strip()).replace(",", "")
    text = re.sub(r'</WR>\w+', 'O', text.strip())
    text = re.sub(r'<WR>', 'WR-B', text.strip())
    text = re.sub(r'WR-B</WR', 'WR-B', text.strip())
    text = re.sub(r'WR-BWR-B', 'WR-B WR-B', text.strip()) 
    text = re.sub(r'WR-B O-B', 'WR-B REP-B', text.strip()) 
    text = re.sub(r'<', 'O', text.strip())
    text = re.sub(r'REP-B?', 'REP-B ', text.strip())
    text = re.sub(r'WR-B?', 'WR-B ', text.strip())
    text = re.sub(r"'\w", 'O', text.strip())
    
    return text

O


### 4단계: 문장 태그 정제 및 공백행 제거

In [336]:
def cleanse_tags(df):
    # 문장 태그 정제
    search_patterns = ['<REP>', '</REP>', '<WR>', '</WR>', '<FIL>', '</FIL>', '?']
    searchtag_patterns = ['?', "'", ',']
    
    for pattern in search_patterns:
        df['Sentence'] = df['Sentence'].str.replace(pattern, '')
        
    for pattern in searchtag_patterns:
        df['Tag'] = df['Tag'].str.replace(pattern, '')

    # 공백이 있는 행 제거
    df = df[df['Sentence'].str.strip() != '']
    
    return df

### 5단계: 데이터 가공 - 학습/테스트 데이터셋 구성

In [337]:
import pandas as pd
from random import shuffle

# 데이터셋 비율 설정
train_ratio = 0.8  # 학습 데이터셋 비율
valid_ratio = 0.1  # 검증 데이터셋 비율

sentences = []

# 문장을 리스트로 추출
for text in main_df['script_tag_txt']:
    sentences.extend(text.split('.'))

# [주의] 랜덤으로 문장 순서 섞도록 설정 가능
#shuffle(sentences)

# 분할 인덱스 계산
train_idx = int(len(sentences) * train_ratio)
valid_idx = int(len(sentences) * (train_ratio + valid_ratio))

# 학습 데이터셋 생성
train_data = []
for sentence in sentences[:train_idx]:
    if sentence.strip():
        train_data.append([sentence.strip(), merge_tags(' '.join(label_text(sentence)))])

# 검증 데이터셋 생성
valid_data = []
for sentence in sentences[train_idx:valid_idx]:
    if sentence.strip():
        valid_data.append([sentence.strip(), merge_tags(' '.join(label_text(sentence)))])
        
# 테스트 데이터셋 생성
test_data = []
for sentence in sentences[valid_idx:]:
    if sentence.strip():
        test_data.append([sentence.strip(), merge_tags(' '.join(label_text(sentence)))])

# 학습 데이터셋을 CSV 파일로 저장
df_train = pd.DataFrame(train_data, columns=["Sentence", "Tag"])
df_train = cleanse_tags(df_train)
# [주의] 데이터전처리 완료된 학습데이터 파일(csv)을 dataset_tagsequence 폴더에 저장되도록 경로설정
df_train.to_csv("tag_train_data_17290e.csv", encoding="utf-8-sig", index=False)

# 검증 데이터셋을 CSV 파일로 저장
df_valid = pd.DataFrame(valid_data, columns=["Sentence", "Tag"])
df_valid = cleanse_tags(df_valid)
# [주의] 데이터전처리 완료된 검증데이터 파일(csv)을 dataset_tagsequence 폴더에 저장되도록 경로설정
df_valid.to_csv("tag_valid_data_2164e.csv", encoding="utf-8-sig", index=False)

# 테스트 데이터셋을 CSV 파일로 저장
df_test = pd.DataFrame(test_data, columns=["Sentence", "Tag"])
df_test = cleanse_tags(df_test)
# [주의] 데이터전처리 완료된 테스트데이터 파일(csv)을 dataset_tagsequence 폴더에 저장되도록 경로설정
df_test.to_csv("tag_test_data_2163e.csv", encoding="utf-8-sig", index=False)


In [338]:
df_train.head()

,Sentence,Tag
0,과학의 발전으로 현대인의 평균 수명이 지속적으로 증가하고 있는 현상을 인류에게 큰 ...,O O O O O O O O WR-B O O O O
1,"오랜 세월 동안 인간의 수명은 다양한 요인에 영향을 받아 변화해왔지만, 현재는 의...",O O O O O O O O O O O O O O O O O O O O O O O O
2,이러한 평균 수명의 연장이 우리의 삶에 어떤 영향을 주는지 살펴보고자 합니다,O O O O O O O O O O O
3,"첫째로, 평균 수명의 연장은 개인의 삶의 질을 향상시킵니다",O O O O O O O O
4,더 오래 살 수 있는 기회는 건강한 삶을 누리는데 큰 도움이 됩니다,O O O O O O O O O O O O


In [339]:
df_valid.head()

,Sentence,Tag
0,어 금융 관리는 우리 삶에서 어 음 중요한 부분을 차지하며 미래를 준비하는...,FIL-B O O O O FIL-B FIL-B O O O O O O O O
1,"어 계획적인 대출, 대출과 카드 사용, 현명한 소비 습관, 금융 계획 수립 등...",FIL-B O REP-B O O O O O O O O O O O O O O F...
2,어 대출과 신용카드 사용은 어 현명 현명하게 계획하고 관리해야 하는 중요한...,FIL-B O O O FIL-B REP-B O O O O O O
3,"금융 저 건강 건강을 유지하려면 신중한 판단과 계획, 책임적인 사용이 어 ...",O REP-B FIL-B O O O O O O O O FIL-B O
4,우리는 자신의 재정 상황을 잘 이해하고 대출과 카드 사용을 어 신중 신중하게 ...,O O O O O O O O O FIL-B REP-B O REP-B WR-B...


In [340]:
df_test.head()

,Sentence,Tag
0,부부로서의 가족 책임과 의무를 피하고 개인적인 목표와 꿈에 집중할 수 있기 때문입니다,O O O O O O O O O O O O
1,경제적인 압박을 덜 받으면서 자신만의 경제적 목표를 추구하며새로운 아이디어와 상업을...,O O O O O O O O O O O O O O O
2,이는 산업발전과 창조적인 활동을 부추길 수 있는 긍정적인 영향을 미칩니다,O O O O O O O O O O
3,비혼은 더 높은 교육 수준과 전문성을 추구할 수 있는 기회를 제공합니다,O O O O O O O O O O O
4,가족의 책임이 줄어들어 시간과 에너지를 자기개발에 투자할 수 있기 때문입니다,O O O O O O O O O O


### 6단계: 학습데이터,검증데이터,테스트데이터 로드(load) 및 데이터전처리

In [341]:
# [주의] 데이터전처리 완료된 학습데이터,검증데이터,테스트데이터 파일(csv)을 dataset_tagsequence 폴더에서 로드하기 위한 경로 설정 필요
train_ner_df = pd.read_csv("tag_train_data_17290e.csv")
valid_ner_df = pd.read_csv("tag_valid_data_2164e.csv")
test_ner_df = pd.read_csv("tag_test_data_2163e.csv")

In [342]:
train_ner_df.head()

,Sentence,Tag
0,과학의 발전으로 현대인의 평균 수명이 지속적으로 증가하고 있는 현상을 인류에게 큰 ...,O O O O O O O O WR-B O O O O
1,"오랜 세월 동안 인간의 수명은 다양한 요인에 영향을 받아 변화해왔지만, 현재는 의...",O O O O O O O O O O O O O O O O O O O O O O O O
2,이러한 평균 수명의 연장이 우리의 삶에 어떤 영향을 주는지 살펴보고자 합니다,O O O O O O O O O O O
3,"첫째로, 평균 수명의 연장은 개인의 삶의 질을 향상시킵니다",O O O O O O O O
4,더 오래 살 수 있는 기회는 건강한 삶을 누리는데 큰 도움이 됩니다,O O O O O O O O O O O O


In [343]:
valid_ner_df.head()

,Sentence,Tag
0,어 금융 관리는 우리 삶에서 어 음 중요한 부분을 차지하며 미래를 준비하는...,FIL-B O O O O FIL-B FIL-B O O O O O O O O
1,"어 계획적인 대출, 대출과 카드 사용, 현명한 소비 습관, 금융 계획 수립 등...",FIL-B O REP-B O O O O O O O O O O O O O O F...
2,어 대출과 신용카드 사용은 어 현명 현명하게 계획하고 관리해야 하는 중요한...,FIL-B O O O FIL-B REP-B O O O O O O
3,"금융 저 건강 건강을 유지하려면 신중한 판단과 계획, 책임적인 사용이 어 ...",O REP-B FIL-B O O O O O O O O FIL-B O
4,우리는 자신의 재정 상황을 잘 이해하고 대출과 카드 사용을 어 신중 신중하게 ...,O O O O O O O O O FIL-B REP-B O REP-B WR-B...


In [344]:
test_ner_df.head()

,Sentence,Tag
0,부부로서의 가족 책임과 의무를 피하고 개인적인 목표와 꿈에 집중할 수 있기 때문입니다,O O O O O O O O O O O O
1,경제적인 압박을 덜 받으면서 자신만의 경제적 목표를 추구하며새로운 아이디어와 상업을...,O O O O O O O O O O O O O O O
2,이는 산업발전과 창조적인 활동을 부추길 수 있는 긍정적인 영향을 미칩니다,O O O O O O O O O O
3,비혼은 더 높은 교육 수준과 전문성을 추구할 수 있는 기회를 제공합니다,O O O O O O O O O O O
4,가족의 책임이 줄어들어 시간과 에너지를 자기개발에 투자할 수 있기 때문입니다,O O O O O O O O O O


In [345]:
print("학습데이터 샘플개수 :", len(train_ner_df))
print("검증데이터 샘플개수 :", len(valid_ner_df))
print("테스트데이터 샘플개수 :", len(test_ner_df))

학습데이터 샘플개수 : 17290
검증데이터 샘플개수 : 2164
테스트데이터 샘플개수 : 2163


In [346]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
valid_data_sentence = [sent.split() for sent in valid_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
valid_data_label = [tag.split() for tag in valid_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

In [347]:
print(train_data_sentence[2])
print(train_data_label[2])

['이러한', '평균', '수명의', '연장이', '우리의', '삶에', '어떤', '영향을', '주는지', '살펴보고자', '합니다']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [348]:
# [주의] 태그가 정의된 파일(ner_label_v2.txt)을 dataset_nerlabel 폴더에서 로드하기 위한 경로 설정 필요
labels = [label.strip() for label in open('ner_label_v2.txt', 'r', encoding='utf-8')]
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['O', 'FIL-B', 'REP-B', 'PS-B', 'WR-B']


In [349]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [350]:
print(tag_to_index)
print(index_to_tag)

{'O': 0, 'FIL-B': 1, 'REP-B': 2, 'PS-B': 3, 'WR-B': 4}
{0: 'O', 1: 'FIL-B', 2: 'REP-B', 3: 'PS-B', 4: 'WR-B'}


In [351]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 5


#### 토크나이저를 통한 형태소 분리 - KLUE-BERT Tokenizer

In [352]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [353]:
sent = train_data_sentence[1]
label = train_data_label[1]
print('문장 :', sent)
print('레이블 :',label)
print('레이블의 정수인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(sent))
print('레이블의 길이 :', len(label))

문장 : ['오랜', '세월', '동안', '인간의', '수명은', '다양한', '요인에', '영향을', '받아', '변화해왔지만,', '현재는', '의료', '기술이', '의료', '기술의', '발전과', '생활환경의', '개선으로', '인해', '평균', '수명이', '급격히', '늘어나고', '있습니다']
레이블 : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
레이블의 정수인코딩 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
문장의 길이 : 24
레이블의 길이 : 24


In [354]:
tokens = []

for one_word in sent:
# 각 단어에 대해서 서브워드로 분리.
# ex) one_word = '쿠 마 리' ===> subword_tokens = ['쿠', '##마 리']
# ex) one_word = '한 동 수 가' ===> subword_tokens = ['한 동', '##수', '##가']
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
print('BERT 토 크 나 이 저 적 용 후 문 장 :',tokens)
print('레이블 :', label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(label))

BERT 토 크 나 이 저 적 용 후 문 장 : ['오랜', '세월', '동안', '인간', '##의', '수명', '##은', '다양', '##한', '요인', '##에', '영향', '##을', '받아', '변화', '##해', '##왔', '##지만', ',', '현재', '##는', '의료', '기술', '##이', '의료', '기술', '##의', '발전', '##과', '생활', '##환경', '##의', '개선', '##으로', '인해', '평균', '수명', '##이', '급격히', '늘어나', '##고', '있', '##습', '##니다']
레이블 : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
레이블의 정수 인코딩 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
문장의 길이 : 44
레이블의 길이 : 24


In [355]:
tokens = []
labels_ids = []
for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  labels_ids.extend([tag_to_index[label_token]]+ [-100] * (len(subword_tokens) -
  1))
print('토큰화 후 문장 :',tokens)
print('레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in
labels_ids])
print('레이블의 정수 인코딩 :', labels_ids)
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(labels_ids))

토큰화 후 문장 : ['오랜', '세월', '동안', '인간', '##의', '수명', '##은', '다양', '##한', '요인', '##에', '영향', '##을', '받아', '변화', '##해', '##왔', '##지만', ',', '현재', '##는', '의료', '기술', '##이', '의료', '기술', '##의', '발전', '##과', '생활', '##환경', '##의', '개선', '##으로', '인해', '평균', '수명', '##이', '급격히', '늘어나', '##고', '있', '##습', '##니다']
레이블 : ['O', 'O', 'O', 'O', '[PAD]', 'O', '[PAD]', 'O', '[PAD]', 'O', '[PAD]', 'O', '[PAD]', 'O', 'O', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'O', '[PAD]', 'O', 'O', '[PAD]', 'O', 'O', '[PAD]', 'O', '[PAD]', 'O', '[PAD]', '[PAD]', 'O', '[PAD]', 'O', 'O', 'O', '[PAD]', 'O', 'O', '[PAD]', 'O', '[PAD]', '[PAD]']
레이블의 정수 인코딩 : [0, 0, 0, 0, -100, 0, -100, 0, -100, 0, -100, 0, -100, 0, 0, -100, -100, -100, -100, 0, -100, 0, 0, -100, 0, 0, -100, 0, -100, 0, -100, -100, 0, -100, 0, 0, 0, -100, 0, 0, -100, 0, -100, -100]
문장의 길이 : 44
레이블의 길이 : 44


In [356]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                                  pad_token_id_for_segment=0,
                                                  pad_token_id_for_label=-100):
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id
  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    tokens = []
    labels_ids = []
    for one_word, label_token in zip(example, label):
      # 하나의 단어에 대해서 서브워드로 토큰화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서브워드 중 첫번째 서브워드만 개체명 레이블을 부여하고 그 외에는 -100으로 채운다.
      labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

    # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대길이를 초과하는 샘플의 경우 max_seq_len - 2의 길이로 변환.
    # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷부분을 자르고 길이 62로 변환.
    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]
    # [SEP]를 추가하는 코드
    # 1. 토큰화 결과의 맨 뒷부분에 [SEP]토큰 추가
    # 2. 레이블에도 맨 뒷부분에 -100 추가.
    tokens += [sep_token]
    labels_ids += [pad_token_id_for_label]

    # [CLS]를 추가하는 코드
    # 1. 토큰화 결과의 앞부분에 [CLS] 토큰 추가
    # 2. 레이블의 맨 앞부분에도 -100 추가.
    tokens = [cls_token] + tokens
    labels_ids = [pad_token_id_for_label] + labels_ids

    # 정수인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    # 어텐션마스크 생성
    attention_mask = [1] * len(input_id)
    # 정수인코딩에 추가할 패딩길이 연산
    padding_count = max_seq_len - len(input_id)
    # 정수인코딩 ,어텐션 마스크에 패딩 추가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)
    # 세그먼트 인코딩.
    token_type_id = [pad_token_id_for_segment] * max_seq_len
    # 레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
    label = labels_ids + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
    assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.asarray(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels



In [357]:
X_train, y_train = convert_examples_to_features(train_data_sentence,
train_data_label, max_seq_len=128, tokenizer=tokenizer)
X_valid, y_valid = convert_examples_to_features(valid_data_sentence, valid_data_label,
max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label,
max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 2163/2163 [00:00<00:00, 2568.41it/s]


In [358]:
print('기존 원문 :', train_data_sentence[0])
print('기존 레이블 :', train_data_label[0])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_train[0][0]])
print('토큰화 후 레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx
in y_train[0]])
print('-' * 50)
print('정수 인코딩 결과 :', X_train[0][0])
print('정수 인코딩 레이블 :', y_train[0])


기존 원문 : ['과학의', '발전으로', '현대인의', '평균', '수명이', '지속적으로', '증가하고', '있는', '현상을', '인류에게', '큰', '변화를', '가져왔습니다']
기존 레이블 : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'WR-B', 'O', 'O', 'O', 'O']
--------------------------------------------------
토큰화 후 원문 : ['[CLS]', '과학', '##의', '발전', '##으로', '현대인', '##의', '평균', '수명', '##이', '지속', '##적으로', '증가', '##하고', '있', '##는', '현상', '##을', '인류', '##에', '##게', '큰', '변화', '##를', '가져왔', '##습', '##니다', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [359]:
print('세그먼트 인코딩 :', X_train[2][0])
print('어텐션 마스크 :', X_train[1][0])

세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [360]:
dropout_rate = 0.1 # 드롭아웃 설정 (기본값: 0.1)

class TFBertForTokenClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForTokenClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.classifier = tf.keras.layers.Dense(num_labels,
                                            kernel_initializer=tf.keras.
                                            initializers.TruncatedNormal
                                            (0.02),
                                            name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
      token_type_ids=token_type_ids)

    # 전체 시퀀스에 대해서 분류해야하므로 outputs[0]임에 주의
    all_output = outputs[0]
    all_output = self.dropout(all_output)  # dropout 레이어 추가
    prediction = self.classifier(all_output)

    return prediction

In [361]:
labels = tf.constant([[-100, 2, 1, -100]])
logits = tf.constant([[[0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15],
  [0.4, 0.5, 0.1]]])

In [362]:
active_loss = tf.reshape(labels, (-1,)) != -100
print(active_loss)

tf.Tensor([False  True  True False], shape=(4,), dtype=bool)


In [363]:
reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])),
  active_loss)
print(reduced_logits)


tf.Tensor(
[[0.06 0.04 0.9 ]
 [0.75 0.1  0.15]], shape=(2, 3), dtype=float32)


In [364]:
labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
print(labels)

tf.Tensor([2 1], shape=(2,), dtype=int32)


In [365]:
def compute_loss(labels, logits):
  # 다중 클래스 분류 문제에서 소프트맥스 함수 미사용시 from_logits=True로 설정.
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  # -100의 값을 가진 정수에 대해서는 오차를 반영하지 않도록 labels를 수정.
  active_loss = tf.reshape(labels, (-1,)) != -100

  # activa_loss로부터 reduced_logits과 labels를 각각 얻는다.
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2]))
    , active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
  return loss_fn(labels, reduced_logits)

### 7단계: 모델 학습 및 평가 - KLUE-BERT Model

In [366]:
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")

In [367]:
with strategy.scope():
  model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size) 
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=compute_loss)


2023-11-13 16:48:53.499662: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFB

In [368]:
class F1score(tf.keras.callbacks.Callback):
  def __init__(self, X_test, y_test):
    self.X_test = X_test
    self.y_test = y_test

  def sequences_to_tags(self, label_ids, pred_ids):
    label_list = []
    pred_list = []

    for i in range(0, len(label_ids)):
      label_tag = []
      pred_tag = []

      # 레이블의 값이 -100인 경우는 F1 score 계산시에도 제외
      # ex) 레이블디코딩과정
      # label_index : [1 -100 2 -100] ===> [1 2] ===> label_tag : [PER-B PER-I]
      for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
        if label_index != -100:
          label_tag.append(index_to_tag[label_index])
          pred_tag.append(index_to_tag[pred_index])

      label_list.append(label_tag)
      pred_list.append(pred_tag)

    return label_list, pred_list

  # 에포크가 끝날때마다 실행되는 함수}
  def on_epoch_end(self, epoch, logs={}):
    y_predicted = self.model.predict(self.X_test)
    y_predicted = np.argmax(y_predicted, axis = 2)

    label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

    score = f1_score(label_list, pred_list, suffix=True)
    print(' - f1: {:04.2f}'.format(score * 100))
    print(classification_report(label_list, pred_list, suffix=True))

### 8단계: 테스트데이터셋 평가(평가지표: F1-Score)

In [369]:
from tensorflow.keras.callbacks import EarlyStopping
import os

# 모델학습 과정에서의 테스트데이터셋 대상 모델 성능 모니터링
f1_score_report = F1score(X_test, y_test)

# 조기 종료
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# 모델 체크포인트
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델 가중치를 저장하는 콜백 
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(
  X_train, y_train, epochs=10, batch_size=32,
  validation_data=(X_valid, y_valid),
  callbacks = [f1_score_report, early_stopping, cp_callback],
)


Epoch 1/10


2023-11-13 16:49:10.569713: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xd4331610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-13 16:49:10.569766: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2023-11-13 16:49:10.569779: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2023-11-13 16:49:10.586339: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-13 16:49:10.821752: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-11-13 16:49:10.997797: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


68/68 [==============================] - 8s 64ms/step
 - f1: 64.45
              precision    recall  f1-score   support

         FIL       0.91      0.89      0.90       831
          PS       0.88      0.88      0.88        16
         REP       0.76      0.78      0.77       848
          WR       0.54      0.18      0.27      1294

   micro avg       0.77      0.55      0.64      2989
   macro avg       0.77      0.68      0.71      2989
weighted avg       0.71      0.55      0.59      2989


Epoch 1: saving model to training_2/cp.ckpt
541/541 [==============================] - 148s 215ms/step - loss: 0.2021 - val_loss: 0.1639
Epoch 2/10
68/68 [==============================] - 5s 73ms/step
 - f1: 63.75
              precision    recall  f1-score   support

         FIL       0.92      0.88      0.90       831
          PS       0.88      0.88      0.88        16
         REP       0.73      0.81      0.77       848
          WR       0.64      0.12      0.20      1294

   micro a

In [370]:
# 모델 체크포인트 파일 확인
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [371]:
# 모델 아키텍처 확인
model.summary()

Model: "tf_bert_for_token_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model (TFBertModel  multiple                  110617344 
 )                                                               
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 110621189 (421.99 MB)
Trainable params: 110621189 (421.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [372]:
# 모델 저장
model.save('saved_model/kluebert_base_new')

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 5), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f54cc054730>, 140006822039824), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 5), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f54cc054730>, 140006822039824), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f54cc06a940>, 140006822042736), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f54cc06a940>, 140006822042736), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 5), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f54cc054730>, 140006822039824), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 5), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f54cc054730>, 140006822039824), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f54cc06a940>, 140006822042736), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(5,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f54cc06a940>, 140006822042736), {}).


INFO:tensorflow:Assets written to: saved_model/kluebert_base_new/assets


INFO:tensorflow:Assets written to: saved_model/kluebert_base_new/assets


In [373]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                                  pad_token_id_for_segment=0,
                                                  pad_token_id_for_label=-100):
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id

  input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

  for example in tqdm(examples):
    tokens = []
    label_mask = []
    for one_word in example:
      # 하나의 단어에 대해서 서브워드로 토큰화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서브워드 중 첫번째 서브워드만 개체명 레이블을 부여하고 그 외에는 -100으로 채운다.
      label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

    # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대길이를 초과하는 샘플의 경우 max_seq_len - 2의 길이로 변환.
    # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변 환.
    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      label_mask = label_mask[:(max_seq_len - special_tokens_count)]

    # [SEP]를 추가하는 코드
    # 1. 토큰화 결과의 맨 뒷부분에 [SEP] 토큰 추가
    # 2. 레이블에도 맨뒷부분에 -100 추가.
    tokens += [sep_token]
    label_mask += [pad_token_id_for_label]

    # [CLS]를 추가하는 코드
    # 1. 토큰화 결과의 앞부분에 [CLS] 토큰 추가
    # 2. 레이블의 맨앞부분에도 -100 추가.
    tokens = [cls_token] + tokens
    label_mask = [pad_token_id_for_label] + label_mask

    # 정수인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)

    # 어텐션 마스크 생성
    attention_mask = [1] * len(input_id)

    # 정수 인코딩에 추가할 패딩 길이 연산
    padding_count = max_seq_len - len(input_id)

    # 정수인코딩 ,어텐션 마스크에 패딩 추가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)

    # 세그먼트 인코딩.
    token_type_id = [pad_token_id_for_segment] * max_seq_len

    # 레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
    label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
    assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    label_masks.append(label_mask)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  label_masks = np.asarray(label_masks, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), label_masks



In [374]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 2191.38it/s]


In [375]:
print('기존 원문 :', test_data_sentence[0])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레이블 마스크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in
label_masks[0]])


기존 원문 : ['부부로서의', '가족', '책임과', '의무를', '피하고', '개인적인', '목표와', '꿈에', '집중할', '수', '있기', '때문입니다']
--------------------------------------------------
토큰화 후 원문 : ['[CLS]', '부부', '##로', '##서', '##의', '가족', '책임', '##과', '의무', '##를', '피하', '##고', '개인', '##적인', '목표', '##와', '꿈', '##에', '집중', '##할', '수', '있', '##기', '때문', '##입니다', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [376]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples,
  max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    # ex) 모델의 예측값 디코딩 과정
    # 예측값(y_predicted)에서 레이블마스크(label_masks)의 값이 -100인 동일 위치의 값을 삭제
    # label_masks : [-100 0 -100 0 -100]
    # y_predicted : [ 0 1 0 2 0 ] ==> [1 2] ==> 최 종 예 측(pred_tag) : [PERB PER-I]
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

### 9단계: 테스트데이터셋 중 샘플 문장 2개 평가 및 검증

In [ ]:
sent1 = '음   셋째로,  익명성은  사회성사생활  침해와 개인정보 유출을  어   가속가속화시킬  수 있습니다'
sent2 = '만약  기침이나 재채기를  <PS/>  기침이나 재채기를  해야 할 경우에는  어  손으로 입과  코  가리고 휴지나 팔꿈치 등으로 치켜서 하여 바이러스의 분사를 최소화해야 합니다'
test_samples = [sent1, sent2]
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)
result_list


100%|██████████| 2/2 [00:00<00:00, 1146.14it/s]
